In [ ]:
# Verify pytorch is installed correctly
import torch
x = torch.rand(5, 3)
print(x)
print(torch.__version__)

## Machine Learning 
### Machine Learning is making the computer learn from studying data and statistics.
### Machine Learning is a step into the direction of artificial intelligence (AI).
### Machine Learning is a program that analyses data and learns to predict the outcome

## Linear regression with Pytorch 
### https://pytorch.org/docs/stable/index.html
### https://towardsdatascience.com/understanding-pytorch-with-an-example-a-step-by-step-tutorial-81fc5f8c4e8e
### https://towardsdatascience.com/linear-regression-with-pytorch-eb6dedead817
### https://www.geeksforgeeks.org/linear-regression-using-pytorch/

In [ ]:
# PyTorch is an optimized tensor library for deep learning using GPUs and CPUs.
# In mathematics, a tensor is an algebraic object that describes a (multilinear) relationship between sets of algebraic objects related to a vector space. 

# Linear Regression is an approach that tries to find a linear relationship 
# between a dependent variable and an independent variable by minimizing the distance



#### 1. Create Data Set, y = 2X +1 

In [ ]:
import numpy as np
# create dummy data for training
x_values = [i for i in range(11)] # zero to 10
x_train = np.array(x_values, dtype=np.float32)
print(x_train)
# reshape with (-1, 1) . We have provided column as 1 but rows as unknown . So we get result new shape as (11, 1).
x_train = x_train.reshape(-1, 1) 
print(x_train)

y_values = [2*i + 1 for i in x_values]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1, 1)

In [ ]:
## Linear Regression with scipy 
import matplotlib.pyplot as plt
from scipy import stats

slope, y_intercept, r, p, std_err = stats.linregress(x_values, y_values)
print(slope,y_intercept)

def myfunc(x):
  return slope * x + y_intercept

mymodel = list(map(myfunc, x_values))

plt.scatter(x_values, y_values)
plt.plot(x_values, mymodel,'r')
plt.show()

#### 2. Define Linear Regression Model inherits PyTorch 

In [ ]:
class linearRegression(torch.nn.Module): # Inherit torch neutral network Module
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__() # Call base class constructor 
        self.linear = torch.nn.Linear(inputSize, outputSize) # Call base class method 

    def forward(self, x):
        out = self.linear(x)
        return out
    
help(torch.nn.Linear)

#### 3. Instantiate the model 

In [ ]:
inputDim = 1        # takes variable 'x' 
outputDim = 1       # takes variable 'y'
learningRate = 0.01 
epochs = 100

model = linearRegression(inputDim, outputDim)

##### For GPU #######
if torch.cuda.is_available():
    model.cuda()

#initialize 
#loss (Mean Squared Error)  
#optimization (Stochastic Gradient Descent) functions that we’ll use in the training of this model.
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

[w,b] = model.parameters()
print(w,b)
w1 = w[0][0].item()
b1 = b[0].item()
print(w1,b1)

#### 4. Train the model 

In [ ]:
from torch.autograd import Variable
losses =[]
for epoch in range(epochs):
    # Converting inputs and labels to Variable
    if torch.cuda.is_available():
        inputs = Variable(torch.from_numpy(x_train).cuda())
        labels = Variable(torch.from_numpy(y_train).cuda())
    else:
        inputs = Variable(torch.from_numpy(x_train))
        labels = Variable(torch.from_numpy(y_train))

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    outputs = model(inputs)

    # get loss for the predicted output
    loss = criterion(outputs, labels)
    losses.append(loss)
    # get gradients w.r.t to parameters
    loss.backward()

    # update parameters
    optimizer.step()

    print('epoch {}, loss {}'.format(epoch, loss.item()))

In [ ]:
#plot losses
plt.plot(range(epochs),losses)
plt.xlabel('epoch')
plt.ylabel('loss')

#### 5. Linear Regression Model is trained, let’s test it

In [ ]:
import matplotlib.pyplot as plt
with torch.no_grad(): # we don't need gradients in the testing phase
    if torch.cuda.is_available():
        predicted = model(torch.from_numpy(x_train).cuda()).cpu().data.numpy()
    else:
        predicted = model(torch.from_numpy(x_train)).data.numpy()
    print(predicted)

plt.clf()
plt.plot(x_train, y_train, 'go', label='True data', alpha=0.5)
plt.plot(x_train, predicted, '--', label='Predictions', alpha=0.5)
plt.legend(loc='best')
plt.show()

#### 6. Test nosie data


In [ ]:
x_sample = torch.randn(100,1)*10
y_sample = x_sample + 3*torch.randn(100,1)
plt.plot(x_sample.numpy(),y_sample.numpy(),'o')
plt.ylabel('y')
plt.xlabel('x')

In [ ]:
with torch.no_grad(): # we don't need gradients in the testing phase
    predicted = model(x_sample).data.numpy()
    print(predicted)

plt.clf()
plt.plot(x_sample, y_sample, 'go', label='True data', alpha=0.5)
plt.plot(x_sample, predicted, '--', label='Predictions', alpha=0.5)
plt.legend(loc='best')
plt.show()